In [242]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [243]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score
%matplotlib inline

In [280]:
X_train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
X_test1 = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
passenger_id = X_test1['PassengerId']

# **Understanding the data**

In [245]:
X_train.shape

In [246]:
X_test1.shape

In [247]:
X_train.head()

In [248]:
X_test1.head()

In [249]:
y_train = X_train['Transported']

In [250]:
X_train.drop(['Transported'],inplace = True,axis = 1)

In [251]:
X_train.info()

In [252]:
X_train.isnull().sum()

In [253]:
## No. of unique values per column
for i in X_train.columns:
    print(f'Column {i} with datatype {X_train[i].dtype} has {X_train[i].nunique()} unique values')

In [254]:
X_train.select_dtypes('object').head(50)

In [255]:
plt.figure(figsize = (10,10))
sns.heatmap(X_train.corr())

In [256]:
X_train.describe()

# **Data Cleaning**

In [257]:
def clean_data(df):
    df[['Cabin_deck','Cabin_num','Cabin_side']] = df['Cabin'].str.split('/',expand = True)
    df[['Passenger_group','Passenger_number']] =  df['PassengerId'].str.split('_',expand = True)
    df['Age'] = pd.cut(df['Age'], bins=[0,14,35,60,110], labels=[0,1,2,3], include_lowest=True)
    df.drop(['Cabin','Passenger_number','Name','PassengerId'],axis = 1,inplace = True)
    return df

In [258]:
X_train.head()

In [259]:
cleaned_train_X = clean_data(X_train)

# **Preprocessing**

In [260]:
'''% of missing data'''
(cleaned_train_X.isnull().sum()/cleaned_train_X.shape[0])*100

In [261]:
def preprocessing(df):
    ord_enc_cols = ['CryoSleep','Cabin_deck','Cabin_side','Passenger_group','VIP']
    hot_enc_cols = ['HomePlanet','Destination']
    impute = SimpleImputer(strategy = 'most_frequent')
    one_enc = OneHotEncoder()
    ord_enc = OrdinalEncoder()
    scale = MinMaxScaler()
    df[df.columns] = impute.fit_transform(df)
    new_hot_cols = one_enc.fit_transform(df[hot_enc_cols]).toarray()
    enc_cols = pd.DataFrame(columns = ['HomePlanet1','HomePlanet2','HomePlanet3','Destination1','Destination2','Destination3'],data = new_hot_cols)
    df = pd.concat([df,enc_cols],axis = 1)
    ord_enc_val = ord_enc.fit_transform(df[ord_enc_cols]) 
    df[ord_enc_cols] = ord_enc_val
    df.drop(hot_enc_cols,inplace = True,axis = 1)
    for col in df.select_dtypes('object'):
        df[col] = df[col].astype('float')
    df[df.columns] = scale.fit_transform(df)
    return df
                            

In [262]:
cleaned_train_X = preprocessing(cleaned_train_X)

In [263]:
plt.figure(figsize = (10,10))
sns.heatmap(cleaned_train_X.corr())

# **Modelling**

In [264]:
X_train,X_test,y_train,y_test = train_test_split(cleaned_train_X,y_train,test_size = 0.2)

In [265]:
models = {'dt_clf':DecisionTreeClassifier(random_state = 42),'svm_clf':SVC(kernel = 'poly',random_state = 42),'rf_clf': RandomForestClassifier(random_state = 42)}

In [266]:
for i in models.keys():
    test_model = models[i].fit(X_train,y_train)
    preds = test_model.predict(X_test)
    print(f'Accuracy for model {i} is {accuracy_score(preds,y_test)}')

In [267]:
params = {'n_estimators':np.arange(200,700,100)}

In [268]:
model = GridSearchCV(RandomForestClassifier(),param_grid=params,verbose = 3,n_jobs = -1)
model.fit(X_train,y_train)
preds = model.predict(X_test)
print(f'Accuracy for final model is {accuracy_score(preds,y_test)}')

In [269]:
test_data = preprocessing(clean_data(X_test1))

In [274]:
preds_final = model.predict(test_data) 


In [281]:
my_submission = pd.DataFrame({'PassengerId': passenger_id, 'Transported': preds_final})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)